In [1]:
import numpy as np
from numba import njit
import time
import matplotlib.pyplot as plt
import json
%run support4.ipynb

In [2]:
# jitted functions
@njit
def _change_state(state,changes):
    new_state = np.copy(state)
    for i,j in enumerate(changes):
        new_state[i] = state[j]
    return new_state

@njit
def _apply_int_moves(state,moves):
    new_state = np.copy(state)
    for move in moves:
        if move == -1: # no move
            break
        new_state = _change_state(new_state, move_array[move])
    return new_state

In [3]:
#             64 65 66 67
#             68 69 70 71
#             72 73 74 75
#             76 77 78 79
# 48 49 50 51  0 1 2 3  16 17 18 19  32 33 34 35
# 52 53 54 55  4 5 6 7  20 21 22 23  36 37 38 39
# 56 57 58 59  8 9 10 11  24 25 26 27  40 41 42 43
# 60 61 62 63  12 13 14 15  28 29 30 31  44 45 46 47
#             80 81 82 83
#             84 85 86 87
#             88 89 90 91
#             92 93 94 95

In [5]:
class Cube4:
    def __init__(self, mode = 0):
        
        # standard mode (0), 1 = front color, 2 = right col, 3 = back col, 4 = left col, 5 = top col, 6 = bottom col
        self.state = np.array([1]*16+[2]*16+[3]*16+[4]*16+[5]*16+[6]*16)
        
        if mode == 1: # if the mode is 1, it shows the indexes instead of colors
            self.state = np.arange(16*6)
            
        if mode == 2: # edge permutations
            self.state = np.array([0]*16*6)
            self.state[1] = self.state[2] = self.state[77] = self.state[78] = 1
            self.state[7] = self.state[11] = self.state[20] = self.state[24] = 2
            self.state[13] = self.state[14] = self.state[81] = self.state[82] = 3
            self.state[4] = self.state[8] = self.state[55] = self.state[59] = 4
            self.state[71] = self.state[75] = self.state[17] = self.state[18] = 5
            self.state[29] = self.state[30] = self.state[87] = self.state[91] = 6
            self.state[61] = self.state[62] = self.state[84] = self.state[88] = 7
            self.state[68] = self.state[72] = self.state[49] = self.state[50] = 8
            self.state[45] = self.state[46] = self.state[93] = self.state[94] = 9
            self.state[39] = self.state[43] = self.state[52] = self.state[56] = 10
            self.state[33] = self.state[34] = self.state[65] = self.state[66] = 11
            self.state[36] = self.state[40] = self.state[23] = self.state[27] = 12
        
        if mode == 3: # edge permutations
            self.state = np.array([0]*16*6)
            self.state[1] = self.state[2] = self.state[20] = self.state[24] = self.state[13] = self.state[14] = self.state[55] = self.state[59] = self.state[17] = self.state[18] = self.state[29] = self.state[30] = self.state[61] = self.state[62] = self.state[49] = self.state[50] = self.state[45] = self.state[46] = self.state[52] = self.state[56] = self.state[33] = self.state[34] = self.state[23] = self.state[27] = 1
    
        if mode == 4: # centers
            self.state = np.array([0]*16*6)
            self.state[5] = self.state[6] = self.state[9] = self.state[10] = 1
            self.state[21] = self.state[22] = self.state[25] = self.state[26] = 2
            self.state[37] = self.state[38] = self.state[41] = self.state[42] = 3
            self.state[53] = self.state[54] = self.state[57] = self.state[58] = 4
            self.state[69] = self.state[70] = self.state[73] = self.state[74] = 5
            self.state[85] = self.state[86] = self.state[89] = self.state[90] = 6

        if mode == 5: # L5E
            self.state = np.array([ 0,  0,  0,  0, -7, -2, -2, -8, -7, -2, -2, -8,  0, -2, -2,  0,  0,
                0,  0,  0, -8, -3, -3,  0, -8, -3, -3,  0,  0, -3, -3,  0,  0,  0,
                0,  0,  0, -4, -4, -6,  0, -4, -4, -6,  0, -4, -4,  0,  0,  0,  0,
                0, -6, -5, -5, -7, -6, -5, -5, -7,  0, -5, -5,  0,  0,  0,  0,  0,
                0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0, -1,
                -1, -1, -1, -1, -1, -1, -1,  0, -1, -1,  0
            ])

        if mode == 6: # L5E
            self.state = np.array([ 0,  0,  0,  0, -7, -2, -2,  0, -7, -2, -2,  0,  0, -2, -2,  0,  0,
                    0,  0,  0,  0, -3, -3, -8,  0, -3, -3, -8,  0, -3, -3,  0,  0,  0,
                    0,  0, -8, -4, -4, -6, -8, -4, -4, -6,  0, -4, -4,  0,  0,  0,  0,
                    0, -6, -5, -5, -7, -6, -5, -5, -7,  0, -5, -5,  0,  0,  0,  0,  0,
                    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0, -1,
                    -1, -1, -1, -1, -1, -1, -1,  0, -1, -1,  0
            ])

        if mode == 7: # LL removed
            self.state = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2,
                    2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3,
                    3, 3, 3, 3, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0,
                    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 6, 6,
                    6, 6, 6, 6, 6, 6, 6, 6
            ])
            

    

    def __repr__(self):
        str_repr = f"\n         {self.state[64]} {self.state[65]} {self.state[66]} {self.state[67]}"
        str_repr += f"\n         {self.state[68]} {self.state[69]} {self.state[70]} {self.state[71]}"
        str_repr += f"\n         {self.state[72]} {self.state[73]} {self.state[74]} {self.state[75]}"
        str_repr += f"\n         {self.state[76]} {self.state[77]} {self.state[78]} {self.state[79]}"
        str_repr += f"\n{self.state[48]} {self.state[49]} {self.state[50]} {self.state[51]}  {self.state[0]} {self.state[1]} {self.state[2]} {self.state[3]}  {self.state[16]} {self.state[17]} {self.state[18]} {self.state[19]}  {self.state[32]} {self.state[33]} {self.state[34]} {self.state[35]}"
        str_repr += f"\n{self.state[52]} {self.state[53]} {self.state[54]} {self.state[55]}  {self.state[4]} {self.state[5]} {self.state[6]} {self.state[7]}  {self.state[20]} {self.state[21]} {self.state[22]} {self.state[23]}  {self.state[36]} {self.state[37]} {self.state[38]} {self.state[39]}"
        str_repr += f"\n{self.state[56]} {self.state[57]} {self.state[58]} {self.state[59]}  {self.state[8]} {self.state[9]} {self.state[10]} {self.state[11]}  {self.state[24]} {self.state[25]} {self.state[26]} {self.state[27]}  {self.state[40]} {self.state[41]} {self.state[42]} {self.state[43]}"
        str_repr += f"\n{self.state[60]} {self.state[61]} {self.state[62]} {self.state[63]}  {self.state[12]} {self.state[13]} {self.state[14]} {self.state[15]}  {self.state[28]} {self.state[29]} {self.state[30]} {self.state[31]}  {self.state[44]} {self.state[45]} {self.state[46]} {self.state[47]}"
        str_repr += f"\n         {self.state[80]} {self.state[81]} {self.state[82]} {self.state[83]}"
        str_repr += f"\n         {self.state[84]} {self.state[85]} {self.state[86]} {self.state[87]}"
        str_repr += f"\n         {self.state[88]} {self.state[89]} {self.state[90]} {self.state[91]}"
        str_repr += f"\n         {self.state[92]} {self.state[93]} {self.state[94]} {self.state[95]}"
        
        return str_repr

    #to be defined
#     def plot(self,colors=["grey","green","r","b","darkorange","w","y"]):
#         plot_2d_cube(self.state, colors=colors) 
    
    def change_state(self,changes):
        self.state = _change_state(self.state,changes)
    
    def apply_moves(self,alg):
        # alg in string
        if alg:
            alg = alg.split(" ")
            for move in alg:
                self.change_state(move_dict[move])

    def apply_int_moves(self,alg):
        # alg is a numpy array of ints
        self.state = _apply_int_moves(self.state,alg)
                
    #to be defined, is nice if you want to apply a specific alg repeatedly
#     def apply_algs(self,alg_list):
#         # moves in string
#         if alg_list:
#             algs = alg_list.split(" ")
#             for alg in algs:
#                 self.change_state(alg_dict[alg])

print("Cube initialized")

cube = Cube4(1)
print(cube)

Cube initialized

         64 65 66 67
         68 69 70 71
         72 73 74 75
         76 77 78 79
48 49 50 51  0 1 2 3  16 17 18 19  32 33 34 35
52 53 54 55  4 5 6 7  20 21 22 23  36 37 38 39
56 57 58 59  8 9 10 11  24 25 26 27  40 41 42 43
60 61 62 63  12 13 14 15  28 29 30 31  44 45 46 47
         80 81 82 83
         84 85 86 87
         88 89 90 91
         92 93 94 95


In [5]:
@njit
def id_from_state(state, ids = np.array(range(96))):
    '''
    assumes no rotation
    make sure the cube is in the correct orientation! (mode 2 for F2L only)
    '''
    # only take the important stickers (0 centers, no cornersticker on top and bottom layer, 3 edge stickers removed)
    ID = str(state[ids[0]])
    for i in ids[1:]:
        ID += str(state[i])
    return ID #you might want to int() this afterwards

In [4]:
'''
Gen parity algs:

give each

'''

'\nGen parity algs:\n\ngive each\n\n'

0.3327779769897461


In [167]:
# cube = Cube4(1)
# cube


         64 65 66 67
         68 69 70 71
         72 73 74 75
         76 77 78 79
48 49 50 51  0 1 2 3  16 17 18 19  32 33 34 35
52 53 54 55  4 5 6 7  20 21 22 23  36 37 38 39
56 57 58 59  8 9 10 11  24 25 26 27  40 41 42 43
60 61 62 63  12 13 14 15  28 29 30 31  44 45 46 47
         80 81 82 83
         84 85 86 87
         88 89 90 91
         92 93 94 95

In [4]:
# def get_centers_id(state):
#     return f"{state[5]}{state[6]}{state[9]}{state[10]}{state[21]}{state[22]}{state[25]}{state[26]}{state[37]}{state[38]}{state[41]}{state[42]}{state[53]}{state[54]}{state[57]}{state[58]}{state[69]}{state[70]}{state[73]}{state[74]}{state[85]}{state[86]}{state[89]}{state[90]}"

# cube = Cube4(4)
# get_centers_id(cube.state)

'111122223333444455556666'

In [5]:
# @njit
# def get_state_id(state,ids = np.array([5,6,9,10,21,22,25,26,37,38,41,42,53,54,57,58,69,70,73,74,85,86,89,90])):
#     ID = ""
#     for i in ids:
#         ID += str(state[i])
#     return ID

# cube = Cube4()
# ids = np.array([5,6,9,10,21,22,25,26,37,38,41,42,53,54,57,58,69,70,73,74,85,86,89,90])
# get_state_id(cube.state)

'111122223333444455556666'

In [6]:
# @njit
# def find_wing(state,cols):
#     '''
#     cubestate in mode 0
#     cols: np array [col1, col2], ex. [1,5]
#     Note: [1,5] and [5,1] are different!
#     '''
#     for pair in np.array([[1,77],[78,2],[7,20],[24,11],[14,82],[81,13],[8,59],[55,4],[17,75],[71,18],[30,91],[87,29],[23,36],[40,27],[33,66],[65,34],[43,56],[52,39],[46,94],[93,45],[49,68],[72,50],[62,84],[88,61]]):
#         if (state[pair[0]],state[pair[1]])==(cols[0],cols[1]):
#             return pair
#     return np.array([0,0])


# cubestate = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])

# find_wing(cubestate, np.array([6,3]))

# t = time.time()
# for i in range(100000):
#     find_wing(cubestate, np.array([6,3]))
# print(time.time()-t)

0.3811347484588623


In [19]:
# @njit
# def get_state_id_advanced(state, centers = np.array([5,6,9,10,21,22,25,26,37,38,41,42,53,54,57,58,69,70,73,74,85,86,89,90]), wings = np.array([[1,6],[6,3],[2,6],[6,2],[1,4],[4,1]])):
#     ID = get_state_id(state,centers)
#     for pair in wings:
#         wing_ids = find_wing(state, pair)
#         ID += str(wing_ids[0])
#     return ID
                          
# get_state_id_advanced(cube.state)

# t = time.time()
# for i in range(100000):
#     find_wing(cube.state, np.array([6,3]))
# print(time.time()-t)

0.340346097946167


In [41]:
# def gen_algs(N):
#     algs = []
#     for n in range(N):
#         t = time.time()
#         n += 1
#         ai = alg_index_special(n)
#         start_alg = ai.get_alg()
#         i = 0
#         while start_alg != ai.get_alg() or i==0:
#             i+=1
#             if ai.ids[0]//3==1:
#                 algs.append(ai.get_alg())
#             ai.increment()
#         print(f"Algs of length {n} ({len(algs)}) genned in {time.time()-t}")
#     return algs

# algs = gen_algs(13)

Algs of length 1 (3) genned in 0.0
Algs of length 2 (12) genned in 0.0
Algs of length 3 (39) genned in 0.0
Algs of length 4 (120) genned in 0.0010013580322265625
Algs of length 5 (363) genned in 0.0009999275207519531
Algs of length 6 (1092) genned in 0.006527900695800781
Algs of length 7 (3279) genned in 0.01953911781311035
Algs of length 8 (9840) genned in 0.0630788803100586
Algs of length 9 (29523) genned in 0.21909213066101074
Algs of length 10 (88572) genned in 0.8858911991119385
Algs of length 11 (265719) genned in 4.071222543716431
Algs of length 12 (797160) genned in 20.473142862319946
Algs of length 13 (2391483) genned in 108.21308827400208


In [17]:
# len(algs)

265719

In [42]:
# table_0 = {
#     # ID: [algs]
# }

# i = 0
# t = time.time()
# for alg in algs:
#     i += 1
#     if i%(len(algs)//100)==0:
#         print(f"{i//(len(algs)//100)}%","-",i,"of",len(algs),"added in", time.time()-t)
#     cube = Cube4()
#     cube.apply_moves(alg)
#     inv = inverse_alg(alg)
    
#     ID = get_state_id_advanced(cube.state)
#     if ID not in table_0.keys():
#         table_0[ID] = [inv]
#     else:
#         table_0[ID] += [inv]
# print("Done")

1% - 23914 of 2391483 added in 4.558806657791138
2% - 47828 of 2391483 added in 9.197020292282104
3% - 71742 of 2391483 added in 13.765568494796753
4% - 95656 of 2391483 added in 18.405800342559814
5% - 119570 of 2391483 added in 22.990396976470947
6% - 143484 of 2391483 added in 27.616947650909424
7% - 167398 of 2391483 added in 32.21114373207092
8% - 191312 of 2391483 added in 36.826284646987915
9% - 215226 of 2391483 added in 41.44115138053894
10% - 239140 of 2391483 added in 46.03905963897705
11% - 263054 of 2391483 added in 50.677534103393555
12% - 286968 of 2391483 added in 55.307700634002686
13% - 310882 of 2391483 added in 59.95593595504761
14% - 334796 of 2391483 added in 64.5928680896759
15% - 358710 of 2391483 added in 69.33339524269104
16% - 382624 of 2391483 added in 74.11867666244507
17% - 406538 of 2391483 added in 78.74999189376831
18% - 430452 of 2391483 added in 83.47668266296387
19% - 454366 of 2391483 added in 88.12347722053528
20% - 478280 of 2391483 added in 92.82

In [43]:
# with open("rU_centers_and_cross_13.json", "w") as outfile:
#     json.dump(table_0, outfile, indent=4)

# # with open("rRU_centers_and_cross_10.json", "r") as outfile:
# #     table_0 = json.load(outfile)

In [44]:
# len(table_0)

532458

In [ ]:
# '''
# To make algs:
# 1. Save all algs consisting of premoves + lookup (=centers preserved)
# 2. Check if edges are solved 
# 3. Check parity
# '''

In [23]:
# @njit
# def edges_solved(state):
#     ''' cube in state 2, checks that the edges are solved (wings are connected) '''
#     pairs = np.array([[1,2],[4,8],[7,11],[13,14],[17,18],[23,27],[29,30],[33,34],[39,43],[45,46],[49,50],[61,62]])
#     for pair in pairs:
#         if state[pair[0]]!=state[pair[1]]:
#             return False
#     return True

# cube = Cube4(2)
# edges_solved(cube.state)

True

In [24]:
# @njit
# def has_parity(state):
#     ''' cube in state 3 '''
#     s = 0
#     for i in [1,20,13,55,17,29,61,49,45,52,33,23]:
#         s = abs(s - state[i])
#     return s
     
# # cube = Cube4(3)
# # cube.state[1]=cube.state[2]=0
# # cube.state[77]=cube.state[78]=1

In [25]:
# def alg_has_parity(alg):
#     parity = 0
#     for move in alg.split(" "):
#         parity += (move[1:]=="w") + (move[1:]=="w'")
#     return parity%2

# def move_types_different(move1, move2):
#     if ("w" in move1 and "w" not in move2) or ("w" in move2 and "w" not in move1):
#         return True
#     if "w" in move1 and "w" in move2 and move1[0]==move2[0]:
#         return False
#     if move1[0]==move2[0]:
#         return False
#     return True

In [214]:
# algs = gen_algs(9)
# len(algs)

Algs of length 1 (3) genned in 0.0
Algs of length 2 (12) genned in 0.0
Algs of length 3 (66) genned in 0.0
Algs of length 4 (309) genned in 0.001999378204345703
Algs of length 5 (1524) genned in 0.011621952056884766
Algs of length 6 (7356) genned in 0.06269693374633789
Algs of length 7 (35787) genned in 0.364335298538208
Algs of length 8 (173568) genned in 2.0712575912475586
Algs of length 9 (842790) genned in 12.200273990631104


842790

In [45]:
# potential_algs = []
# i = 0
# t = time.time()
# for pre in algs:
#     i += 1
#     if i%(len(algs)//100)==0:
#         print(f"{i//(len(algs)//100)}%","-",i,"of",len(algs),"in",time.time()-t)
#     cube = Cube4()
#     cube.apply_moves(pre)
#     ID = get_state_id_advanced(cube.state)
#     if ID in table_0.keys():
#         for suf in table_0[ID]:
#             pot_alg = pre + " " + suf
#             if pot_alg and alg_has_parity(pot_alg) and move_types_different(pre.split(" ")[-1], suf.split(" ")[0]):
#                 potential_algs.append(pot_alg)

1% - 23914 of 2391483 in 6.993045806884766
2% - 47828 of 2391483 in 13.133779287338257
3% - 71742 of 2391483 in 19.082365036010742
4% - 95656 of 2391483 in 25.119628429412842
5% - 119570 of 2391483 in 31.18677043914795
6% - 143484 of 2391483 in 37.37446212768555
7% - 167398 of 2391483 in 43.5600221157074
8% - 191312 of 2391483 in 49.587300062179565
9% - 215226 of 2391483 in 55.7418475151062
10% - 239140 of 2391483 in 61.77702355384827
11% - 263054 of 2391483 in 67.83035492897034
12% - 286968 of 2391483 in 73.47173404693604
13% - 310882 of 2391483 in 79.01221251487732
14% - 334796 of 2391483 in 84.47723507881165
15% - 358710 of 2391483 in 90.0939781665802
16% - 382624 of 2391483 in 95.56571054458618
17% - 406538 of 2391483 in 101.27522397041321
18% - 430452 of 2391483 in 107.10228371620178
19% - 454366 of 2391483 in 112.83329105377197
20% - 478280 of 2391483 in 118.36173224449158
21% - 502194 of 2391483 in 123.84860920906067
22% - 526108 of 2391483 in 129.501314163208
23% - 550022 of 23

In [34]:
# len(potential_algs)

167368

In [46]:
# parity_algs = []

# i = 0
# t = time.time()
# for alg in potential_algs:
    
#     i += 1
#     if i%(len(potential_algs)//100)==0:
#         print(f"{i//(len(potential_algs)//100)}%","-",i,"of",len(potential_algs),"in",time.time()-t)
    
#     cube2 = Cube4(2)
#     cube2.apply_moves(alg.strip())
    
#     # check if edges solved
#     if edges_solved(cube2.state) and alg not in parity_algs:
#         parity_algs.append(alg)
#         print("OLL PARITY ALG:",alg)

1% - 10183 of 1018372 in 0.31533002853393555
2% - 20366 of 1018372 in 0.6369478702545166
3% - 30549 of 1018372 in 0.9649407863616943
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U'
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U2
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U'
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U2
OLL PARITY ALG: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U'
OLL PARITY ALG: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U
OLL PARITY ALG: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U2
OLL PARITY ALG: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw

8% - 81464 of 1018372 in 2.6662750244140625
OLL PARITY ALG: U Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U'
OLL PARITY ALG: U Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U
OLL PARITY ALG: U Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U2
OLL PARITY ALG: U' Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U'
OLL PARITY ALG: U' Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U
OLL PARITY ALG: U' Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U2
OLL PARITY ALG: U2 Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U'
OLL PARITY ALG: U2 Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U
OLL PARITY ALG: U2 Rw U' Rw' U2 Rw U2 Rw' U' Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U2 Rw' U2
9% - 91647 of 1018372 in 3.002884864807129
10% - 101830 of 1018372 in 3.340331554412842
11% - 112013 of 1018372 in 3.6786439418792725


26% - 264758 of 1018372 in 8.91173791885376
OLL PARITY ALG: U Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U'
OLL PARITY ALG: U Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U
OLL PARITY ALG: U Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U2
OLL PARITY ALG: U' Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U'
OLL PARITY ALG: U' Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U
OLL PARITY ALG: U' Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U2
OLL PARITY ALG: U2 Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U'
OLL PARITY ALG: U2 Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U
OLL PARITY ALG: U2 Rw U Rw2 U' Rw2 U Rw U2 Rw' U2 Rw U2 Rw U2 Rw' U Rw U' Rw' U' Rw2 U2 Rw' U2
OLL PARITY ALG: U Rw' U2 Rw2 U2 Rw U' Rw U2 Rw' U2 Rw U' Rw2 U' Rw U2 Rw' U2 Rw U Rw2 U2 Rw U'
OLL PARITY A

29% - 295307 of 1018372 in 9.965068101882935
OLL PARITY ALG: U Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U'
OLL PARITY ALG: U Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U
OLL PARITY ALG: U Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U2
OLL PARITY ALG: U' Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U'
OLL PARITY ALG: U' Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U
OLL PARITY ALG: U' Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U2
OLL PARITY ALG: U2 Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U'
OLL PARITY ALG: U2 Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U
OLL PARITY ALG: U2 Rw U Rw2 U Rw2 U2 Rw2 U Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U2
OLL PARITY ALG: U Rw U' Rw2 U2 Rw2 U Rw2 U' Rw2 U2 Rw U2 Rw' U2 Rw U' Rw U' Rw' U' Rw2 U2 Rw' U'


35% - 356405 of 1018372 in 12.092174530029297
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U'
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U2
OLL PARITY ALG: U' Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U'
OLL PARITY ALG: U' Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U
OLL PARITY ALG: U' Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U2
OLL PARITY ALG: U2 Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U'
OLL PARITY ALG: U2 Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U
OLL PARITY ALG: U2 Rw' U2 Rw2 U' Rw2 U Rw2 U2 Rw2 U' Rw' U2 Rw U2 Rw' U Rw' U Rw U Rw2 U2 Rw U2
36% - 366588 of 1018372 in 12.44402551651001
OLL PARITY ALG: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 

OLL PARITY ALG: U Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U'
OLL PARITY ALG: U Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U
OLL PARITY ALG: U Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U2
OLL PARITY ALG: U' Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U'
OLL PARITY ALG: U' Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U
OLL PARITY ALG: U' Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U2
OLL PARITY ALG: U2 Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U'
OLL PARITY ALG: U2 Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U
OLL PARITY ALG: U2 Rw U2 Rw' U2 Rw' U' Rw2 U2 Rw U2 Rw' U2 Rw' U2 Rw2 U2 Rw' U2 Rw2 U2 Rw U' Rw U2
40% - 407320 of 1018372 in 13.854782819747925
OLL PARITY ALG: U Rw U2 Rw2 U' Rw' U' Rw U' Rw' U2 Rw' U2 Rw2 U2 Rw' 

OLL PARITY ALG: U Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U'
OLL PARITY ALG: U Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U
OLL PARITY ALG: U Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U2
OLL PARITY ALG: U' Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U'
OLL PARITY ALG: U' Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U
OLL PARITY ALG: U' Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U2
OLL PARITY ALG: U2 Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U'
OLL PARITY ALG: U2 Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U
OLL PARITY ALG: U2 Rw U Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U Rw' U2 Rw' U2 Rw U2
OLL PARITY ALG: U Rw U' Rw U2 Rw2 U2 Rw' U2 Rw2 U2 Rw' U2 Rw' U2 Rw U2 Rw2 U' Rw' U2 Rw' U2 Rw U'
OLL PARITY ALG: U Rw U' Rw U2 Rw2 U

OLL PARITY ALG: U Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U'
OLL PARITY ALG: U Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U
OLL PARITY ALG: U Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U2
OLL PARITY ALG: U' Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U'
OLL PARITY ALG: U' Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U
OLL PARITY ALG: U' Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U2
OLL PARITY ALG: U2 Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U'
OLL PARITY ALG: U2 Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U
OLL PARITY ALG: U2 Rw' U' Rw' U2 Rw U2 Rw' U Rw U' Rw2 U2 Rw' U2 Rw U2 Rw2 U Rw U2 Rw2 U2 Rw' U2
49% - 498967 of 1018372 in 17.02246594429016
OLL PARITY ALG: U Rw U2 Rw2 U Rw U2 Rw' U2 Rw U' Rw2 U' Rw U2 Rw' U2 Rw U' Rw U2 Rw2 U2 

52% - 529516 of 1018372 in 18.08084011077881
OLL PARITY ALG: U Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U'
OLL PARITY ALG: U Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U
OLL PARITY ALG: U Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U2
OLL PARITY ALG: U' Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U'
OLL PARITY ALG: U' Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U
OLL PARITY ALG: U' Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U2
OLL PARITY ALG: U2 Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U'
OLL PARITY ALG: U2 Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U
OLL PARITY ALG: U2 Rw' U2 Rw' U Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw2 U' Rw2 U Rw' U2 Rw U2 Rw' U' Rw U2
OLL PARITY ALG: U Rw U Rw2 U2 Rw' U2 Rw U2 Rw U2 Rw2 U2 Rw U2 Rw2 U2 Rw' U Rw' U2 Rw' U2

58% - 590614 of 1018372 in 20.196765184402466
59% - 600797 of 1018372 in 20.549596548080444
60% - 610980 of 1018372 in 20.89945077896118
61% - 621163 of 1018372 in 21.26995611190796
62% - 631346 of 1018372 in 21.621790885925293
63% - 641529 of 1018372 in 21.972105979919434
64% - 651712 of 1018372 in 22.32498860359192
65% - 661895 of 1018372 in 22.676870822906494
66% - 672078 of 1018372 in 23.02797842025757
67% - 682261 of 1018372 in 23.379916191101074
68% - 692444 of 1018372 in 23.72977375984192
69% - 702627 of 1018372 in 24.081642389297485
70% - 712810 of 1018372 in 24.43147325515747
71% - 722993 of 1018372 in 24.782978534698486
72% - 733176 of 1018372 in 25.135849714279175
73% - 743359 of 1018372 in 25.48721742630005
74% - 753542 of 1018372 in 25.84004521369934
75% - 763725 of 1018372 in 26.192288160324097
76% - 773908 of 1018372 in 26.552311182022095
77% - 784091 of 1018372 in 26.902778387069702
78% - 794274 of 1018372 in 27.277785778045654
79% - 804457 of 1018372 in 27.627618789672

In [50]:
# len(parity_algs)

936

In [49]:
# for i,a in enumerate(parity_algs):
#     print(f"{i+1}: {a}")

1: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U'
2: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U
3: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U2
4: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U'
5: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U
6: U Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U2
7: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U'
8: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U
9: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U' Rw2 U Rw U2
10: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U'
11: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U
12: U' Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw U' Rw2 U Rw2 U' Rw2 U2 Rw' U2
13: U2 Rw' U2 Rw2 U' Rw' U' Rw U' Rw U2 Rw' U2 Rw' U2 Rw2 U Rw2 U

In [51]:
# parity_algs

[]

In [51]:
# i = 1
# for alg in parity_algs:
#     cube = Cube4()
#     cube.apply_moves(alg)
#     state = cube.state
#     if state[7]==state[11]==state[15]==1 and state[20]==state[24]==state[28]==state[31]==state[27]==state[23]==2 and state[36]==state[40]==state[44]==3:
#         print(i,"-",alg)
#         i+=1

In [39]:
# p_algs = []

# for alg in parity_algs:
#     p_alg = alg[2:-2].strip()
#     if p_alg not in p_algs:
#         p_algs.append(p_alg)